In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Directorios originales
img_dir = r'C:\CatFLW dataset\images'
labels_dir = r'C:\CatFLW dataset\labels'

# Directorios de destino
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Crear directorios de destino si no existen
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Obtener lista de archivos de imágenes
image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

# Dividir en entrenamiento y validación (80% - 20%)
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Función para copiar archivos
def copy_files(file_list, src_img_dir, src_labels_dir, dest_img_dir, dest_labels_dir):
    for file_name in file_list:
        shutil.copy(os.path.join(src_img_dir, file_name), os.path.join(dest_img_dir, file_name))
        label_name = file_name.replace('.png', '.json')
        shutil.copy(os.path.join(src_labels_dir, label_name), os.path.join(dest_labels_dir, label_name))

# Copiar archivos de entrenamiento
copy_files(train_files, img_dir, labels_dir, train_img_dir, train_labels_dir)

# Copiar archivos de validación
copy_files(val_files, img_dir, labels_dir, val_img_dir, val_labels_dir)

print("Archivos divididos y copiados exitosamente.")

Archivos divididos y copiados exitosamente.


In [2]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
oreja_izq = [27, 28, 29, 30, 31]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EarLeftDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_oreja_izq = calcular_centro_region(landmarks, oreja_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_oreja_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la oreja izquierda
        ear_left_points = np.array([landmarks[i] for i in oreja_izq], dtype=np.float32)
        ear_left_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(ear_left_points), 1))
        points_ones = np.hstack([ear_left_points, ones])
        ear_left_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la oreja izquierda
        centro_x, centro_y = calcular_centro_region(ear_left_points_rotated, range(len(oreja_izq)))

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_ear = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_ear, (224, 224))

        # Ajustar los puntos faciales de la oreja izquierda al nuevo recorte
        ear_left_points_adjusted = [(p[0] - x1, p[1] - y1) for p in ear_left_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        ear_left_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in ear_left_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=ear_left_points_scaled)
            image = augmented['image']
            ear_left_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(ear_left_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EarLeftDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EarLeftDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EarLeft(nn.Module):
    def __init__(self):
        super(DenseNet121EarLeft, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(oreja_izq) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EarLeft().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para oreja izquierda)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, ear_left_points, _, _, _ in train_dataloader:
            images, ear_left_points = images.cuda(), ear_left_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, ear_left_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_ear_left_points, _, _, _ in val_dataloader:
                    val_images, val_ear_left_points = val_images.cuda(), val_ear_left_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_ear_left_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_ear_left_points.cpu().numpy().reshape(-1, len(oreja_izq), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(oreja_izq), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(oreja_izq))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EarLeftLandmarks V2.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` 

Epoch [1/300], Loss: 17954.9333
Validation Loss: 15458.0908
NME: 1.4174
Best model saved at epoch 1 with val_loss 15458.0908
Epoch [2/300], Loss: 9575.6536
Validation Loss: 3547.8036
NME: 0.6036
Best model saved at epoch 2 with val_loss 3547.8036
Epoch [3/300], Loss: 2145.3656
Validation Loss: 1493.0034
NME: 0.3450
Best model saved at epoch 3 with val_loss 1493.0034
Epoch [4/300], Loss: 1642.2495
Validation Loss: 1402.8177
NME: 0.3424
Best model saved at epoch 4 with val_loss 1402.8177
Epoch [5/300], Loss: 1591.5684
Validation Loss: 1368.2254
NME: 0.3445
Best model saved at epoch 5 with val_loss 1368.2254
Epoch [6/300], Loss: 1558.3180
Validation Loss: 1349.9158
NME: 0.3421
Best model saved at epoch 6 with val_loss 1349.9158
Epoch [7/300], Loss: 1545.3773
Validation Loss: 1324.5729
NME: 0.3350
Best model saved at epoch 7 with val_loss 1324.5729
Epoch [8/300], Loss: 1536.6972
Validation Loss: 1293.7827
NME: 0.3307
Best model saved at epoch 8 with val_loss 1293.7827
Epoch [9/300], Loss: 

In [4]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
oreja_der = [22, 23, 24, 25, 26]  # Cambiado a oreja derecha

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EarRightDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_oreja_der = calcular_centro_region(landmarks, oreja_der)  # Cambiado a oreja derecha
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_oreja_der -= np.array([x_min, y_min])  # Cambiado a oreja derecha
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la oreja derecha
        ear_right_points = np.array([landmarks[i] for i in oreja_der], dtype=np.float32)  # Cambiado a oreja derecha
        ear_right_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(ear_right_points), 1))
        points_ones = np.hstack([ear_right_points, ones])
        ear_right_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la oreja derecha
        centro_x, centro_y = calcular_centro_region(ear_right_points_rotated, range(len(oreja_der)))  # Cambiado a oreja derecha

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_ear = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_ear, (224, 224))

        # Ajustar los puntos faciales de la oreja derecha al nuevo recorte
        ear_right_points_adjusted = [(p[0] - x1, p[1] - y1) for p in ear_right_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        ear_right_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in ear_right_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=ear_right_points_scaled)
            image = augmented['image']
            ear_right_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(ear_right_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EarRightDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EarRightDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EarRight(nn.Module):
    def __init__(self):
        super(DenseNet121EarRight, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(oreja_der) * 2)  # Cambiado a oreja derecha
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EarRight().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para oreja derecha)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, ear_right_points, _, _, _ in train_dataloader:  # Cambiado a oreja derecha
            images, ear_right_points = images.cuda(), ear_right_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, ear_right_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_ear_right_points, _, _, _ in val_dataloader:  # Cambiado a oreja derecha
                    val_images, val_ear_right_points = val_images.cuda(), val_ear_right_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_ear_right_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_ear_right_points.cpu().numpy().reshape(-1, len(oreja_der), 2))  # Cambiado a oreja derecha
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(oreja_der), 2))  # Cambiado a oreja derecha

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(oreja_der))  # Cambiado a oreja derecha
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EarRightLandmarks V2.pth')  # Cambiado a oreja derecha
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

Epoch [1/300], Loss: 15668.7481
Validation Loss: 13677.4727
NME: 1.1790
Best model saved at epoch 1 with val_loss 13677.4727
Epoch [2/300], Loss: 8574.2104
Validation Loss: 3927.0695
NME: 0.5627
Best model saved at epoch 2 with val_loss 3927.0695
Epoch [3/300], Loss: 2143.7017
Validation Loss: 1310.6911
NME: 0.2688
Best model saved at epoch 3 with val_loss 1310.6911
Epoch [4/300], Loss: 1344.7225
Validation Loss: 1147.6663
NME: 0.2661
Best model saved at epoch 4 with val_loss 1147.6663
Epoch [5/300], Loss: 1284.9474
Validation Loss: 1095.6877
NME: 0.2677
Best model saved at epoch 5 with val_loss 1095.6877
Epoch [6/300], Loss: 1262.4926
Validation Loss: 1087.1326
NME: 0.2678
Best model saved at epoch 6 with val_loss 1087.1326
Epoch [7/300], Loss: 1240.9988
Validation Loss: 1072.7133
NME: 0.2524
Best model saved at epoch 7 with val_loss 1072.7133
Epoch [8/300], Loss: 1209.9801
Validation Loss: 1029.8412
NME: 0.2564
Best model saved at epoch 8 with val_loss 1029.8412
Epoch [9/300], Loss: 

In [1]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EyeLeftDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave del ojo izquierdo
        eye_left_points = np.array([landmarks[i] for i in ojo_izq], dtype=np.float32)
        eye_left_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(eye_left_points), 1))
        points_ones = np.hstack([eye_left_points, ones])
        eye_left_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado del ojo izquierdo
        centro_x, centro_y = calcular_centro_region(eye_left_points_rotated, range(len(ojo_izq)))

        # Definir los límites del recorte de 56x56 píxeles
        half_crop_size = 28
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_eye = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_eye, (224, 224))

        # Ajustar los puntos faciales del ojo izquierdo al nuevo recorte
        eye_left_points_adjusted = [(p[0] - x1, p[1] - y1) for p in eye_left_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        eye_left_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in eye_left_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=eye_left_points_scaled)
            image = augmented['image']
            eye_left_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(eye_left_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EyeLeftDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EyeLeftDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EyeLeft(nn.Module):
    def __init__(self):
        super(DenseNet121EyeLeft, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(ojo_izq) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EyeLeft().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para ojo izquierdo)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, eye_left_points, _, _, _ in train_dataloader:
            images, eye_left_points = images.cuda(), eye_left_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, eye_left_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_eye_left_points, _, _, _ in val_dataloader:
                    val_images, val_eye_left_points = val_images.cuda(), val_eye_left_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_eye_left_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_eye_left_points.cpu().numpy().reshape(-1, len(ojo_izq), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(ojo_izq), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(ojo_izq))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EyeLeftLandmarks V2.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` 

Epoch [1/300], Loss: 14254.8805
Validation Loss: 12328.7725
NME: 6.8208
Best model saved at epoch 1 with val_loss 12328.7725
Epoch [2/300], Loss: 7159.9013
Validation Loss: 2474.5272
NME: 2.4992
Best model saved at epoch 2 with val_loss 2474.5272
Epoch [3/300], Loss: 1371.1002
Validation Loss: 874.0512
NME: 1.1930
Best model saved at epoch 3 with val_loss 874.0512
Epoch [4/300], Loss: 778.6536
Validation Loss: 645.4695
NME: 1.0322
Best model saved at epoch 4 with val_loss 645.4695
Epoch [5/300], Loss: 656.3076
Validation Loss: 579.3322
NME: 1.0196
Best model saved at epoch 5 with val_loss 579.3322
Epoch [6/300], Loss: 625.2623
Validation Loss: 566.0329
NME: 1.0107
Best model saved at epoch 6 with val_loss 566.0329
Epoch [7/300], Loss: 620.6083
Validation Loss: 557.9990
NME: 1.0220
Best model saved at epoch 7 with val_loss 557.9990
Epoch [8/300], Loss: 614.4968
Validation Loss: 557.1649
NME: 1.0200
Best model saved at epoch 8 with val_loss 557.1649
Epoch [9/300], Loss: 611.4090
Validati

In [2]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EyeRightDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave del ojo derecho
        eye_right_points = np.array([landmarks[i] for i in ojo_der], dtype=np.float32)
        eye_right_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(eye_right_points), 1))
        points_ones = np.hstack([eye_right_points, ones])
        eye_right_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado del ojo derecho
        centro_x, centro_y = calcular_centro_region(eye_right_points_rotated, range(len(ojo_der)))

        # Definir los límites del recorte de 56x56 píxeles
        half_crop_size = 28
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_eye = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_eye, (224, 224))

        # Ajustar los puntos faciales del ojo derecho al nuevo recorte
        eye_right_points_adjusted = [(p[0] - x1, p[1] - y1) for p in eye_right_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        eye_right_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in eye_right_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=eye_right_points_scaled)
            image = augmented['image']
            eye_right_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(eye_right_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EyeRightDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EyeRightDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EyeRight(nn.Module):
    def __init__(self):
        super(DenseNet121EyeRight, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(ojo_der) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EyeRight().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para ojo derecho)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, eye_right_points, _, _, _ in train_dataloader:
            images, eye_right_points = images.cuda(), eye_right_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, eye_right_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_eye_right_points, _, _, _ in val_dataloader:
                    val_images, val_eye_right_points = val_images.cuda(), val_eye_right_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_eye_right_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_eye_right_points.cpu().numpy().reshape(-1, len(ojo_der), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(ojo_der), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(ojo_der))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EyeRightLandmarks.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

Epoch [1/300], Loss: 14212.2814
Validation Loss: 12039.6662
NME: 6.7084
Best model saved at epoch 1 with val_loss 12039.6662
Epoch [2/300], Loss: 6811.4053
Validation Loss: 2062.4994
NME: 2.2246
Best model saved at epoch 2 with val_loss 2062.4994
Epoch [3/300], Loss: 1115.2185
Validation Loss: 680.4316
NME: 1.1018
Best model saved at epoch 3 with val_loss 680.4316
Epoch [4/300], Loss: 675.0620
Validation Loss: 579.1433
NME: 1.0557
Best model saved at epoch 4 with val_loss 579.1433
Epoch [5/300], Loss: 648.3628
Validation Loss: 579.7174
NME: 1.0577
Epoch [6/300], Loss: 646.2025
Validation Loss: 568.0584
NME: 1.0353
Best model saved at epoch 6 with val_loss 568.0584
Epoch [7/300], Loss: 635.3362
Validation Loss: 561.9599
NME: 1.0271
Best model saved at epoch 7 with val_loss 561.9599
Epoch [8/300], Loss: 625.6166
Validation Loss: 549.0863
NME: 1.0458
Best model saved at epoch 8 with val_loss 549.0863
Epoch [9/300], Loss: 623.0666
Validation Loss: 540.1474
NME: 1.0057
Best model saved at e

In [3]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
nariz = [0, 2, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 32, 33, 34, 35, 42, 43, 44, 45, 46, 47]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class NoseDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos y la nariz
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_nariz = calcular_centro_region(landmarks, nariz)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_nariz -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la nariz
        nose_points = np.array([landmarks[i] for i in nariz], dtype=np.float32)
        nose_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(nose_points), 1))
        points_ones = np.hstack([nose_points, ones])
        nose_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la nariz
        centro_x, centro_y = calcular_centro_region(nose_points_rotated, range(len(nariz)))

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_nose = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_nose, (224, 224))

        # Ajustar los puntos faciales de la nariz al nuevo recorte
        nose_points_adjusted = [(p[0] - x1, p[1] - y1) for p in nose_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        nose_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in nose_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=nose_points_scaled)
            image = augmented['image']
            nose_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(nose_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = NoseDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = NoseDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121Nose(nn.Module):
    def __init__(self):
        super(DenseNet121Nose, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(nariz) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121Nose().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para nariz)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, nose_points, _, _, _ in train_dataloader:
            images, nose_points = images.cuda(), nose_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, nose_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_nose_points, _, _, _ in val_dataloader:
                    val_images, val_nose_points = val_images.cuda(), val_nose_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_nose_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_nose_points.cpu().numpy().reshape(-1, len(nariz), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(nariz), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(nariz))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'NoseAreaLandmarks V2.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

Epoch [1/300], Loss: 16295.4488
Validation Loss: 14465.2864
NME: 3.2240
Best model saved at epoch 1 with val_loss 14465.2864
Epoch [2/300], Loss: 9046.4421
Validation Loss: 3420.5417
NME: 1.1890
Best model saved at epoch 2 with val_loss 3420.5417
Epoch [3/300], Loss: 1968.8574
Validation Loss: 1269.3468
NME: 0.6183
Best model saved at epoch 3 with val_loss 1269.3468
Epoch [4/300], Loss: 1095.9607
Validation Loss: 927.7681
NME: 0.5398
Best model saved at epoch 4 with val_loss 927.7681
Epoch [5/300], Loss: 945.4137
Validation Loss: 865.4155
NME: 0.5401
Best model saved at epoch 5 with val_loss 865.4155
Epoch [6/300], Loss: 921.8355
Validation Loss: 859.3690
NME: 0.5429
Best model saved at epoch 6 with val_loss 859.3690
Epoch [7/300], Loss: 903.0185
Validation Loss: 843.0697
NME: 0.5533
Best model saved at epoch 7 with val_loss 843.0697
Epoch [8/300], Loss: 904.0369
Validation Loss: 837.3348
NME: 0.5458
Best model saved at epoch 8 with val_loss 837.3348
Epoch [9/300], Loss: 896.6160
Valid